In [ ]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
import matplotlib.pyplot as plt

import uproot
import pickle
import nue_booster
import importlib
importlib.reload(nue_booster)

In [ ]:
fold = "nuselection"
tree = "NeutrinoSelectionFilter"

PATH = '/home/david/data/searchingfornues/v08_00_00_25/cc0pinp/1013/'

#nc = uproot.open(PATH+"prodgenie_nc_pi0_uboone_overlay-v08_00_00_16_run1_reco2.root")[fold][tree]
mc = uproot.open(PATH+"prodgenie_bnb_nu_uboone_overlay_mcc9.1_run3_G_reco2.root")[fold][tree]
nue = uproot.open(PATH+"prodgenie_bnb_intrinsic_nue_uboone_overlay_mcc9.1_run3_G_reco2.root")[fold][tree]
ext = uproot.open(PATH+"data_extbnb_mcc9.1_v08_00_00_16_18_run3_G_reco2.root")[fold][tree]

variables = nue_booster.variables.copy()
variables.remove("is_signal")
variables.remove("train_weight")
variables.append("category")
variables.append("selected")

#nc = nc.pandas.df(variables + ["weightSpline"], flatten=False)
mc = mc.pandas.df(variables + ["weightSpline"], flatten=False)
nue = nue.pandas.df(variables + ["weightSpline"], flatten=False)
ext = ext.pandas.df(variables, flatten=False)

ext["weightSpline"] = 1
#nc["is_signal"] = nc["category"] == 11
nue["is_signal"] = nue["category"] == 11
mc["is_signal"] = mc["category"] == 11
ext["is_signal"] = ext["category"] == 11

#nc["reco_e"] = (nc["shr_energy_tot_cali"] + 0.03) / 0.79 + nc["trk_energy_tot"]
nue["reco_e"] = (nue["shr_energy_tot_cali"] + 0.03) / 0.79 + nue["trk_energy_tot"]
mc["reco_e"] = (mc["shr_energy_tot_cali"] + 0.03) / 0.79 + mc["trk_energy_tot"]
ext["reco_e"] = (ext["shr_energy_tot_cali"] + 0.03) / 0.79 + ext["trk_energy_tot"]

In [ ]:
lee_bins = [0, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.6, 0.8]
lee_scaling = [
    6.3744101, 6.3744101, 5.6455402, 3.7305500, 1.5091400, 1.0742800, 0.7540929,
    0.4763070, 0.1523270
]
#lee_scaling = [1,1,1,1,1,1,1,1,1]

#nc["train_weight"] = 0
nue["train_weight"] = 0
mc["train_weight"] = 0
ext["train_weight"] = 0

for i, lee_bin in enumerate(lee_bins):
    
    if i == 0:
        continue
        
    nue.loc[(nue['reco_e'] > lee_bins[i-1]) & (nue['reco_e'] < lee_bins[i]), 'train_weight'] = lee_scaling[i-1] * nue['weightSpline']
    mc.loc[(mc['reco_e'] > lee_bins[i-1]) & (mc['reco_e'] < lee_bins[i]), 'train_weight'] = lee_scaling[i-1] * mc['weightSpline']
    ext.loc[(ext['reco_e'] > lee_bins[i-1]) & (ext['reco_e'] < lee_bins[i]), 'train_weight'] = lee_scaling[i-1] * ext['weightSpline']
    #nc.loc[(nc['reco_e'] > lee_bins[i-1]) & (nc['reco_e'] < lee_bins[i]), 'train_weight'] = lee_scaling[i-1] * nc['weightSpline']

In [ ]:
train_mc, test_mc = train_test_split(mc, test_size=0.5, random_state=1990)
train_ext, test_ext = train_test_split(ext, test_size=0.5, random_state=1990)
train_nue, test_nue = train_test_split(nue, test_size=0.5, random_state=1990)
#train_nc, test_nc = train_test_split(nc, test_size=0.5, random_state=1990)

samples = {
    "mc": (train_mc, test_mc),
    "nue": (train_nue, test_nue),
    "ext": (train_ext, test_ext)#,
    #"nc": (train_nc, test_nc)
} 

fig, ax = plt.subplots(1,1)

my_booster = nue_booster.NueBooster(samples, nue_booster.variables, random_state=1990)
my_booster.set_preselection("reco_e < 1.0 and selected == 1")

for label, bkg_query in zip(nue_booster.labels, nue_booster.bkg_queries):
    
    preds = my_booster.train_booster(ax, bkg_query)
    
    with open(PATH+'pickles/booster_%s.pickle' % label, 'wb') as booster_file:
        pickle.dump(preds, booster_file)

    variables = nue_booster.variables.copy()
    variables.remove("is_signal")
    variables.remove("nu_e")
    variables.remove("train_weight")
#     variables.remove('shr_energy_tot_cali')
#     variables.remove('trk_energy_tot')
        
    mc_prediction = preds.predict(
        xgb.DMatrix(mc[variables]),
        ntree_limit=preds.best_iteration + 1)
    nue_prediction = preds.predict(
        xgb.DMatrix(nue[variables]),
        ntree_limit=preds.best_iteration + 1)
    ext_prediction = preds.predict(
        xgb.DMatrix(ext[variables]),
        ntree_limit=preds.best_iteration + 1)
    #nc_prediction = preds.predict(
    #    xgb.DMatrix(nc[variables]),
    #    ntree_limit=preds.best_iteration + 1)

    #nc["%s_score" % label] = nc_prediction
    mc["%s_score" % label] = mc_prediction
    nue["%s_score" % label] = nue_prediction
    ext["%s_score" % label] = ext_prediction


ax.set_ylim([0, 1.05])
ax.set_xlim([0, 1.0])
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC')
ax.legend()
fig.tight_layout()
fig.savefig("plots/roc_single.pdf")

In [ ]:
train_mc, test_mc = train_test_split(mc, test_size=0.5, random_state=1990)
train_ext, test_ext = train_test_split(ext, test_size=0.5, random_state=1990)
train_nue, test_nue = train_test_split(nue, test_size=0.5, random_state=1990)
#train_nc, test_nc = train_test_split(nc, test_size=0.5, random_state=1990)

samples = {
    "mc": (train_mc, test_mc),
    "nue": (train_nue, test_nue),
    "ext": (train_ext, test_ext),
    #"nc": (train_nc, test_nc)
} 

variables_bdts = [
    "is_signal", "nu_e", "train_weight", "ext_score", "ncpi0_score", "cc_score",
    "ccpi0_score", "cosmic_score"
]

In [ ]:
fig_global, ax_global = plt.subplots(1, 1)
my_global_booster = nue_booster.NueBooster(samples, variables_bdts, 1990)

preds = my_global_booster.train_booster(ax_global)

ax_global.legend()
ax_global.set_xlabel('False Positive Rate')
ax_global.set_ylabel('True Positive Rate')
ax_global.set_title('ROC')
fig_global.tight_layout()
fig_global.savefig(PATH+"plots/roc_global.pdf")

In [ ]:
with open(PATH+'pickles/booster.pickle', 'wb') as booster_file:
    pickle.dump(preds, booster_file)